In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import gc
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output
seed = 2019

/kaggle/input/wns-analyticswizard/test_aq1fgdb/test.csv
/kaggle/input/wns-analyticswizard/train_na17sgz/view_log.csv
/kaggle/input/wns-analyticswizard/train_na17sgz/item_data.csv
/kaggle/input/wns-analyticswizard/train_na17sgz/train.csv
/kaggle/input/wns-analyticswizard/sample_submission_ipsblct/sample_submission.csv


In [2]:
train = pd.read_csv('../input/wns-analyticswizard/train_na17sgz/train.csv', parse_dates = ['impression_time'])
view_log = pd.read_csv('../input/wns-analyticswizard/train_na17sgz/view_log.csv', parse_dates = ['server_time'])
item_data = pd.read_csv('../input/wns-analyticswizard/train_na17sgz/item_data.csv')
test = pd.read_csv('../input/wns-analyticswizard/test_aq1fgdb/test.csv', parse_dates = ['impression_time'])

In [3]:
# train = train.sort_values(by='impression_time')
# view_log = view_log.sort_values(by='server_time')
# test = test.sort_values(by='impression_time')

In [4]:
train.head()

,impression_id,impression_time,user_id,app_code,os_version,is_4G,is_click
0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,87862,422,old,0,0
1,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,63410,467,latest,1,1
2,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,71748,259,intermediate,1,0
3,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,69209,244,latest,1,0
4,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,62873,473,latest,0,0


In [5]:
train['month'] = train['impression_time'].dt.month
train['date'] = train['impression_time'].dt.day
train['hour'] = train['impression_time'].dt.hour
train['minute'] = train['impression_time'].dt.minute

test['month'] = test['impression_time'].dt.month
test['date'] = test['impression_time'].dt.day
test['hour'] = test['impression_time'].dt.hour
test['minute'] = test['impression_time'].dt.minute

In [6]:
ntrain= train.shape[0]
df = pd.concat([train, test], ignore_index= True, axis=0)

In [7]:
ad_iter = np.zeros(len(df))

for user in df['user_id'].unique():
    indices = df[df['user_id'] == user].index
    for i in range(len(indices)):
        ad_iter[indices[i]] = i + 1

In [8]:
df['ad_iter'] = pd.Series(ad_iter)

In [9]:
df['ad_delta'] = df.groupby('user_id')['impression_time'].diff().dt.total_seconds()/60

In [10]:
# X = pd.get_dummies(train['os_version'])
# train = train.drop('os_version',1)
# train = pd.concat([train, X], axis=1)

# X = pd.get_dummies(test['os_version'])
# test = test.drop('os_version',1)
# test = pd.concat([test, X], axis=1)

In [11]:
view_log.head()

,server_time,device_type,session_id,user_id,item_id
0,2018-10-15 08:58:00,android,112333,4557,32970
1,2018-10-15 08:58:00,android,503590,74788,7640
2,2018-10-15 08:58:00,android,573960,23628,128855
3,2018-10-15 08:58:00,android,121691,2430,12774
4,2018-10-15 08:58:00,android,218564,19227,28296


In [12]:
df['item_id'] = -1

In [13]:
for user in df['user_id'].unique():
    X = df[df['user_id']==user]['impression_time']
    X_log = view_log[view_log['user_id']==user][['server_time','item_id']]
    for time in X:
        index = X[X == time].index
        valid = X_log[X_log['server_time'] < time].index
        if len(valid) > 0:
            item = X_log['item_id'][valid[-1]]
            df['item_id'][index] = item
        else:
            continue

In [14]:
df['time_elapsed'] = -1

In [15]:
for user in df['user_id'].unique():
    X = df[df['user_id']==user]['impression_time']
    X_log = view_log[view_log['user_id']==user]['server_time']
    for time in X:
        index = X[X == time].index
        valid = list(X_log[X_log < time])
        if len(valid) > 0:
            time_elapsed = (time - valid[-1]).total_seconds()/60
            df['time_elapsed'][index] = time_elapsed
        else:
            continue

In [16]:
df['total_visits'] = -1

In [17]:
for user in df['user_id'].unique():
    X = df[df['user_id']==user][['impression_time','item_id']]
    X_log = view_log[view_log['user_id']==user][['server_time','item_id']]
    for index,row in X.iterrows():
        valid = X_log[(X_log['server_time'] < row['impression_time']) & (X_log['item_id']== row['item_id'])]
        if len(valid) > 0:
            df['total_visits'][index] = len(valid)
        else:
            continue

In [18]:
# train['total_visits'] = -1

In [19]:
# for user in train['user_id'].unique():
#     X = train[train['user_id']==user][['impression_time','item_id']]
#     X_log = view_log[view_log['user_id']==user][['server_time','item_id']]
#     for index,row in X.iterrows():
# #         index = X[X['impression_time'] == time].index
#         valid = X_log[(X_log['server_time'] < row['impression_time']) & (X_log['item_id']== row['item_id'])]
#         if len(valid) > 0:
#             train['total_visits'][index] = len(valid)
#         else:
#             continue

In [20]:
df.head()

,app_code,date,hour,impression_id,impression_time,is_4G,is_click,minute,month,os_version,user_id,ad_iter,ad_delta,item_id,time_elapsed,total_visits
0,422,15,0,c4ca4238a0b923820dcc509a6f75849b,2018-11-15 00:00:00,0,0.0,0,11,old,87862,1.0,NaN,43886,37789.0,1
1,467,15,0,45c48cce2e2d7fbdea1afc51c7c6ad26,2018-11-15 00:01:00,1,1.0,1,11,latest,63410,1.0,NaN,43209,29938.0,5
2,259,15,0,70efdf2ec9b086079795c442636b55fb,2018-11-15 00:02:00,1,0.0,2,11,intermediate,71748,1.0,NaN,122348,32343.0,1
3,244,15,0,8e296a067a37563370ded05f5a3bf3ec,2018-11-15 00:02:00,1,0.0,2,11,latest,69209,1.0,NaN,104035,301.0,1
4,473,15,0,182be0c5cdcd5072bb1864cdee4d3d6e,2018-11-15 00:02:00,0,0.0,2,11,latest,62873,1.0,NaN,4069,140.0,1


In [21]:
df = df.replace(-1, np.nan)

In [22]:
train = df.iloc[:ntrain, :]
test = df.iloc[ntrain:, :]

if 'is_click' in test.columns:
    test = test.drop('is_click',1)

In [23]:
print(ntrain)
print(train.shape)
print(test.shape)

237609
(237609, 16)
(90675, 15)


In [24]:
train.to_csv('wns_train.csv', index=False)
test.to_csv('wns_test.csv', index=False)

In [25]:
# target = train['is_click']
# train = train.drop(['impression_id','impression_time','is_click'],1)

# test = test.drop(['impression_id','impression_time','is_click'], 1)

In [26]:
# def group_noise(col):
#     valid = train[col].value_counts()
#     valid = valid[valid>10].index
    
#     train[col] = np.where(train[col].isin(valid),train[col],np.nan)
#     test[col] = np.where(test[col].isin(valid),test[col],np.nan)
    
#     return train[col], test[col]

In [27]:
# train['ad_iter'], test['ad_iter'] = group_noise('ad_iter')

In [28]:
# n_folds = 5
# skfold = StratifiedKFold(n_splits = n_folds, random_state = seed)
# kfold = KFold(n_splits = n_folds, random_state = seed)

# oof = np.zeros(len(train))
# pred = np.zeros(len(test))
# scores=[]
# feat_imp = pd.DataFrame()
# feat_imp['features'] = train.columns
# feat_imp['importance'] = 0

In [29]:


# for fold_n, (train_index, valid_index) in enumerate(skfold.split(train, target)):
    
#     print('Fold ',fold_n + 1)
    
#     xgbclf = XGBClassifier(
#         n_estimators=1000,
#         max_depth=3,
#         learning_rate=0.1,
#         subsample=1,
#         colsample_bytree=1,
#         random_state = seed
# #         tree_method='gpu_hist'  # THE MAGICAL PARAMETER
#     )
    
#     X_train_, X_valid = train.iloc[train_index], train.iloc[valid_index]
#     y_train_, y_valid = target.iloc[train_index], target.iloc[valid_index]
#     xgbclf.fit(X_train_,y_train_, early_stopping_rounds = 50, eval_set=[(X_train_,y_train_),(X_valid, y_valid)], eval_metric = 'auc', verbose = 50)
#     del X_train_,y_train_
#     pred += xgbclf.predict_proba(test)[:,1]/n_folds
#     val=xgbclf.predict_proba(X_valid)[:,1]
#     oof[valid_index] = val
#     scores.append(roc_auc_score(y_valid, val))
#     feat_imp['importance'] += xgbclf.feature_importances_/n_folds
#     del xgbclf, X_valid
#     print('ROC accuracy: {}'.format(roc_auc_score(y_valid, val)))
#     del val,y_valid
#     gc.collect()

# print('Mean CV ', np.mean(scores))

In [30]:
# sub = pd.read_csv('../input/wns-analyticswizard/sample_submission_ipsblct/sample_submission.csv')
# sub['is_click'] = pd.Series(pred)
# sub.to_csv('second_sub.csv', index=False)

In [31]:
# feat_imp.to_csv('Feat_imp_sub1.csv',index=False)

In [32]:
# df=feat_imp.sort_values(by='importance', ascending=False)
# plt.figure(figsize=(10,10))
# sns.barplot(data= df, x=df['importance'], y=df['features'])
# plt.title('Feature Importances')